In [1]:
import geopandas as gpd
import pandas as pd

## Open State Data

In [2]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)
states_df = states_df.to_crs(9311)

## Open Population Data

In [3]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [4]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [5]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

## Open Aviation Facilities Data

In [6]:
file_path = "data/NTAD_Aviation_Facilities_-4386985993802120360.gpkg"
aviation_gdf = gpd.read_file(file_path)

In [7]:
heliport_gdf = aviation_gdf[aviation_gdf["SITE_TYPE_CODE"] == "H"].reset_index()
heliport_gdf = heliport_gdf.to_crs(9311)
heliport_gdf.to_file("data/heliports.gpkg")

In [8]:
heliport_counts_df = (
    pd.DataFrame(heliport_gdf.groupby("STATE_CODE").size())
    .reset_index()
    .rename(columns={"STATE_CODE": "STUSPS", 0: "HELIPORTS"})
)

## Merge data

In [9]:
heliport_counts_gdf = states_with_population_df[
    ["NAME", "STUSPS", "geometry", "POPULATION"]
].merge(heliport_counts_df, on="STUSPS", how="inner")

In [10]:
heliport_counts_gdf["sqmi"] = heliport_counts_gdf["geometry"].area / 2589988.11
heliport_counts_gdf["HELIPORTS_PER_10_SQMI"] = heliport_counts_gdf["HELIPORTS"] / (
    heliport_counts_gdf["sqmi"] / 10.0
)

In [11]:
heliport_counts_gdf["per_100k"] = (
    (heliport_counts_gdf["HELIPORTS"] / (heliport_counts_gdf["POPULATION"] / 100_000))
    .round(decimals=2)
    .astype(float)
)

## Export

In [12]:
heliport_counts_gdf = heliport_counts_gdf.dropna()
heliport_counts_gdf.to_file("data/heliports_per_state.gpkg")